In [1]:
#import everything
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import urllib2
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from bs4.diagnose import diagnose
from lxml import etree
import requests
import bs4

# Fetch world population statistics from Wikipedia
**In this excercise, Wikipedia's population statistics will be parsed, cleaned and analysed.

    1) Import the html and create a BS tree

In [2]:
url = ('https://en.wikipedia.org/wiki/List_of_countries_by_past,_current_and_future_population')
def get_site_html(url):
    source = urllib2.urlopen(url).read()
    return source
def get_tree(url):
    source = get_site_html(url)
    soup = BeautifulSoup(source, 'lxml')
    return soup
soup = get_tree(url)

Extract the data from the tables and fill a DataFrame

In [5]:
names = ['Country', '1950', '1960', '1965', '1970', '1975', '1980']
tabulated_values = []
for td in soup.find_all('tr'):
    try:
        tabulated_values.append(td.stripped_strings)
    except:
        pass    
mydata = DataFrame(tabulated_values)#, index=None, columns=names) #reads data into pandas dataframe
mydata = mydata.ix[0:686] #truncates irrelevant rows
mydata = mydata.drop(mydata.columns[14:],axis=1)
mydata = mydata.drop([0,4,6,8,10,12], axis=1)
#rename the column headers
mydata.rename(columns={1:'Country',2:'1950', 3:'1955', 5:'1960', 7:'1965', 9:'1970', 11:'1975', 13:'1980'}, inplace=True)
mydata = mydata.ix[1:686]
# set the country names as the index for plotting 
mydata = mydata.set_index('Country')
mydata.head(3)

,1950,1955,1960,1965,1970,1975,1980
Country,,,,,,,
Afghanistan,"8,151","8,892","9,830","10,998","12,431","14,133","15,045"
Albania,"1,228","1,393","1,624","1,884","2,157","2,402","2,672"
Algeria,"8,893","9,842","10,910","11,964","13,932","16,141","18,807"


## Convert the unicode values to 'float'

In [7]:
def convert_to_float(x):
    try:
        # Replace commas and convert to float
        return float(x.replace(',', ''))
    except:
        # There is some cell with a % sign in it, so just return instead of failing
        return x

# Apply the above function to every cell via applymap
mydata = mydata.applymap(convert_to_float)
mydata.head(3)

,1950,1955,1960,1965,1970,1975,1980
Country,,,,,,,
Afghanistan,8151,8892,9830,10998,12431,14133,15045
Albania,1228,1393,1624,1884,2157,2402,2672
Algeria,8893,9842,10910,11964,13932,16141,18807


In [9]:
row = mydata.ix['Albania']
print row
plt.plot(row.index, row, label=row.name) 

         1950  1955  1960  1965  1970  1975  1980
Country                                          
Albania  1228  1393  1624  1884  2157  2402  2672
Albania  2957  2.05  1.88 -0.54     0 -0.86 -0.25
Albania  3075   0.3   0.2 -0.01 -0.26 -0.45 -0.55


AttributeError: 'DataFrame' object has no attribute 'name'

In [24]:
plotCountries = ['Austria', 'Germany', 'United States', 'France']
    
for country in plotCountries:
    row = mydata.ix[country]
    row.plot()
#     plt.plot(row) 
    
plt.ylim(ymin=0) # start y axis at 0

plt.xticks(rotation=70)
plt.legend(loc='best')
plt.xlabel("Year")
plt.ylabel("# people (million)")
plt.title("Population of countries")
plt.show()